In [2]:
from Bio.PDB import PDBList
import os
import requests
import pandas as pd
import subprocess
import time
import tempfile

In [3]:
df_test = pd.read_csv('./pair_sample.csv', index_col=0)
df_test

,meso_pid,thermo_pid,meso_pdb,thermo_pdb
0,P9WJA3,A0A1M6N9Z6,1Y5H,NaN
1,I6XFS7,A0A1M6WSV2,6M1C,NaN
2,Q65EQ1,A0A521F3Z2,6NKG,NaN
3,F5HRS7,A0A2T0LBQ2,7QH4,NaN
4,P9WHM1,C7MUW2,3LP6,NaN
...,...,...,...,...
95,A0A4Z0GXN3,A0A1G7W5M9,NaN,NaN
96,G4H893,A0A1W6VMF1,NaN,NaN
97,A0A120GMI5,Q5L0I9,NaN,NaN
98,A0A4V2YRI4,A0A3N2GW27,NaN,NaN


In [5]:
def download_structures(df, pdb_column, u_column, pdb_dir):
    start_time = time.time()  # Start measuring time
    pdbl = PDBList()
    if not os.path.exists(pdb_dir):
        os.makedirs(pdb_dir)
        
    for i, row in df.iterrows():
        pdb_id = row[pdb_column]
        uniprot_id = row[u_column]
        if not pd.isna(pdb_id):  # check for NaN value in PDB IDs column
            pdbl.retrieve_pdb_file(pdb_id, pdir=pdb_dir, file_format='pdb')
            file_path = os.path.join(pdb_dir, f'pdb{pdb_id.lower()}.ent')
            if os.path.exists(file_path):
                os.rename(os.path.join(file_path), os.path.join(pdb_dir, f'{pdb_id}.pdb'))
            else:
                pass
        elif isinstance(uniprot_id, str):  # download structure using UniProt ID
            url = f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb'
            response = requests.get(url)
            if response.ok:
                filename = f'{pdb_dir}/{uniprot_id}.pdb'
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded file for {uniprot_id} to {filename}")
            else:
                print(f"Failed to download file for {uniprot_id}: {response.status_code} - {response.reason}")
        else:
            print(f"No PDB ID or UniProt ID available for index {i}")
        end_time = time.time()  # Stop measuring time
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")
    pass

In [6]:
download_structures(df=df_test, pdb_column='meso_pdb', u_column='meso_pid', pdb_dir='checking')

Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Desired structure doesn't exist
Downloaded file for I4BFW2 to checking/I4BFW2.pdb
Downloaded file for A0A562LEY4 to checking/A0A562LEY4.pdb
Downloaded file for A0A1E3SPQ4 to checking/A0A1E3SPQ4.pdb
Downloaded file for A0A366XZE1 to checking/A0A366XZE1.pdb
Downloaded file for K6BYX8 to checking/K6BYX8.pdb
Downloaded file for A0A1M7ZPM6 to checking/A0A1M7ZPM6.pdb
Downloaded file for A0A3M8BW34 to checking/A0A3M8BW34.pdb
Downloaded file for A0A1G6TTW2 to checking/A0A1G6TTW2.pdb
Downloaded file for A0A7W6BCG0 to checking/A0A7W6BCG0.pdb
Downloaded file for A0A3S9ZD69 to checking/A0A3S9ZD69.pdb
Downloaded file for A0A150H7B5 to checking/A0A150H7B

In [7]:
download_structures(df=df_test, pdb_column='thermo_pdb', u_column='thermo_pid', pdb_dir='checking')

Downloaded file for A0A1M6N9Z6 to checking/A0A1M6N9Z6.pdb
Downloaded file for A0A1M6WSV2 to checking/A0A1M6WSV2.pdb
Downloaded file for A0A521F3Z2 to checking/A0A521F3Z2.pdb
Downloaded file for A0A2T0LBQ2 to checking/A0A2T0LBQ2.pdb
Downloaded file for C7MUW2 to checking/C7MUW2.pdb
Downloaded file for A0A7W9YPC6 to checking/A0A7W9YPC6.pdb
Downloaded file for A0A087LCG0 to checking/A0A087LCG0.pdb
Downloaded file for I3DYT5 to checking/I3DYT5.pdb
Downloaded file for A0A853ALZ0 to checking/A0A853ALZ0.pdb
Downloaded file for A0A521CL06 to checking/A0A521CL06.pdb
Downloaded file for A0A3N2H419 to checking/A0A3N2H419.pdb
Downloaded file for A0A7V9Z9X5 to checking/A0A7V9Z9X5.pdb
Downloaded file for A0A1G6PFE9 to checking/A0A1G6PFE9.pdb
Downloaded file for A0A4R8LSA2 to checking/A0A4R8LSA2.pdb
Downloaded file for A0A7W3RA64 to checking/A0A7W3RA64.pdb
Downloaded file for A0A540V080 to checking/A0A540V080.pdb
Downloaded file for A0A2G8B7Q0 to checking/A0A2G8B7Q0.pdb
Downloaded file for A0A7W8MUD5

In [1]:
def run_fatcat(df, pdb_dir):
    p_values = []  # List to store the extracted p-values
    rows_to_drop = []  # List to store the indices of rows to be dropped

    for index, row in df.iterrows():
        if not pd.isna(row['meso_pdb']):
            p1 = row['meso_pdb']
        else:
            p1 = row['meso_pid']
        
        if not pd.isna(row['thermo_pdb']):
            p2 = row['thermo_pdb']
        else:
            p2 = row['thermo_pid']
        
        # Check if the structure files exist in the 'checking' folder
        p1_file = f'{p1}.pdb'
        p2_file = f'{p2}.pdb'
        if not os.path.exists(os.path.join(pdb_dir, p1_file)) or not os.path.exists(os.path.join(pdb_dir, p2_file)):
            # Append the index of the row to the list of rows to be dropped
            rows_to_drop.append(index)
            continue

        # Set the FATCAT command and its arguments
        cmd = ['FATCAT', '-p1', p1_file, '-p2', p2_file, '-i', pdb_dir, '-q']
        
        # Run the FATCAT command and capture the output
        result = subprocess.run(cmd, capture_output=True, text=True)
        output = result.stdout

        # Find the line containing the p-value
        p_value_line = next(line for line in output.split('\n') if line.startswith("P-value"))

        # Extract the p-value
        p_value = float(p_value_line.split()[1])
        
        # Append the p-value to the list
        p_values.append(p_value)

    # Drop the rows with missing structure files from the dataframe
    df = df.drop(rows_to_drop)
    
    df.loc[:, 'p_value'] = p_values  # Use .loc to set the 'p_value' column
    return df

In [4]:
df_result_new = run_fatcat(df_test, pdb_dir='checking')

In [5]:
df_result_new

,meso_pid,thermo_pid,meso_pdb,thermo_pdb,p_value
0,P9WJA3,A0A1M6N9Z6,1Y5H,NaN,7.180000e-13
1,I6XFS7,A0A1M6WSV2,6M1C,NaN,0.000000e+00
2,Q65EQ1,A0A521F3Z2,6NKG,NaN,9.500000e-12
4,P9WHM1,C7MUW2,3LP6,NaN,0.000000e+00
5,Q5FJB3,A0A7W9YPC6,3TFX,NaN,0.000000e+00
...,...,...,...,...,...
95,A0A4Z0GXN3,A0A1G7W5M9,NaN,NaN,0.000000e+00
96,G4H893,A0A1W6VMF1,NaN,NaN,0.000000e+00
97,A0A120GMI5,Q5L0I9,NaN,NaN,0.000000e+00
98,A0A4V2YRI4,A0A3N2GW27,NaN,NaN,2.470000e-12


In [16]:
df_result_new['p_value'] = pd.to_numeric(df_result_new['p_value'], errors='coerce')

# Create a new dataframe with rows where p-value > 0.05
new_df_new = df_result_new[df_result_new['p_value'] > 0.05]
new_df_new


,meso_pid,thermo_pid,meso_pdb,thermo_pdb,p_value
22,P66054,A0A4R3N1Y2,8A57,NaN,0.0871
25,P66125,A0A540V080,8A63,NaN,0.1010


In [ ]:
# Quick fatcat command: 
# FATCATQue.pl timeused pair.list -q >pair.aln (only work when files are in the same directory)
# FATCAT -p1 A0JNW5.pdb -p2 A1L1K8.pdb -o A0JNW5_A1L1K8 -m
# FATCAT -p1 P0A9P0.pdb -p2 Q9H3D4.pdb -i ./checking -o test -m (-i directory of PDB files), result file is in ./

#FATCAT command help: 
FATCAT <-p1 file> <-p2 file> (the input pdb files)
  [-o output-initial] (default tmp)
  [-i string] (data directory for both structures, default ./)
  [-i1 string] (data directory for 1st structures, default ./)
  [-i2 string] (data directory for 2st structures, default ./)
  [-s1 num] (read start position of protein 1, default from the begin)
  [-s2 num] (read start position of protein 2, default from the begin)
  [-l1 num] (read length of protein 1, default whole protein)
  [-l2 num] (read length of protein 2, default whole protein)
  [-r] (force program run rigid structural alignment, default off)
  [-filter probcut] (filter the alignment quickly, set a big probcut, eg 0.2, useful in database searching, default off)
  [-sparse number[0-3]] (sparsely fragment sampling, for speeding up the calculation, default off)
  [-b] (print a basic report to stdout)
  [-f] (print a full report to stdout. When -b or -f is on, following options are all automatically off)
  [-m] (print alignment to a file)
  [-q] (print alignment to stdout, useful in database-search in queue)
  [-ab] (print the postscript graph of all AFPs and final AFP chain in black-white to a file)
  [-ac] (print the postscript graph of all AFPs and final AFP chain in color to a file)
  [-c] (print AFP chaining result to file.chain.txt)
  [-t] (print the files of transformed pdbs and corresponding rasmol scripts)
  [-s] (print the files of superimposed pdbs and corresponding rasmol scripts)
  [-time] (print the total running time, default off)

In [ ]:
def run_fatcat(df, pdb_dir):
    p_values = []  # List to store the extracted p-values
    rows_to_drop = []  # List to store the indices of rows to be dropped

    for index, row in df.iterrows():
        if not pd.isna(row['meso_pdb']):
            p1 = row['meso_pdb']
        else:
            p1 = row['meso_pid']
        
        if not pd.isna(row['thermo_pdb']):
            p2 = row['thermo_pdb']
        else:
            p2 = row['thermo_pid']
        
        # Check if the structure files exist in the 'checking' folder
        p1_file = f'{p1}.pdb'
        p2_file = f'{p2}.pdb'
        if not os.path.exists(os.path.join(pdb_dir, p1_file)) or not os.path.exists(os.path.join(pdb_dir, p2_file)):
            # Append the index of the row to the list of rows to be dropped
            rows_to_drop.append(index)
            continue

        # Set the FATCAT command and its arguments
        cmd = ['FATCAT', '-p1', p1_file, '-p2', p2_file, '-i', pdb_dir, '-q']
        
        # Run the FATCAT command and capture the output
        result = subprocess.run(cmd, capture_output=True, text=True)
        output = result.stdout

        # Find the line containing the p-value
        p_value_line = next(line for line in output.split('\n') if line.startswith("P-value"))

        # Extract the p-value
        p_value = float(p_value_line.split()[1])
        
        # Append the p-value to the list
        p_values.append(p_value)

    # Drop the rows with missing structure files from the dataframe
    df = df.drop(rows_to_drop)
    
    df.loc[:, 'p_value'] = p_values  # Use .loc to set the 'p_value' column
    return df